# JDH's Quantum Computing Notes 
Simulations of basic quantum computation logic and algorithms in Python.

## Contents
- Basic logic and gates
- Measurement and vizualization (todo)
- Deutsch–Jozsa (todo)

## Basic Logic

### Initialization
A quantum circuit class with a handful of implemented gates. Qubits are stored and addressed in little-endian order, but printed in big-endian (for now)

Initial state: $|00..\rangle = |0\rangle \otimes |0\rangle ...$

In [45]:
import functools as ft
import numpy as np

class QCircuit:
    def __init__(self, num_qubits: int):
        assert num_qubits > 0, "The number of qubits should be greater than zero."
        self.num_qubits = num_qubits
        self.state = ft.reduce(lambda x, y: np.kron(x, y), [np.array([1.0, 0.0], dtype = complex)] * num_qubits)
        
    def __str__(self):
        return np.array_str(self.state)
        
# Create system with two qubits in the zero state and output the associated state vector
qc = QCircuit(2)
assert np.allclose(qc.state, np.array([1, 0, 0, 0])), "State vector not initialized properly"

### Single qubit gates
Gate matrices are calculated on each method call before being applied to the current state vector.

$ NOT_1 = I \otimes X \otimes I $

$ M|000\rangle = |010\rangle $

In [42]:
class QCircuit(QCircuit):
    def compose(self, operations: list[tuple[int, np.array]]):
        output = [np.identity(2)] * self.num_qubits
        for operation in operations:
            assert operation[0] < self.num_qubits, "Qubit index out of range."
            output[operation[0]] = operation[1]
        return ft.reduce(lambda x, y: np.kron(x, y), reversed(output))
        
    def px(self, target: int):
        operation = [(target, np.array([[0, 1], [1, 0]]))]
        gate = self.compose(operation)
        self.state = np.dot(gate, self.state)

    def h(self, target: int):
        operation = [(target, (1 / np.sqrt(2)) * np.array([[1, 1], [1, -1]]))]
        gate = self.compose(operation)
        self.state = np.dot(gate, self.state)

# Apply the Hadamard gate on qubit zero in |00>
qc = QCircuit(2)
qc.h(0)
assert np.allclose(qc.state, np.array([0.70710678, 0.70710678, 0, 0])), "Improper Hadamard functionality"

# Apply the Pauli-X (NOT) gate on the second qubit in the |000> system
qc = QCircuit(3)
qc.px(1)
assert np.allclose(qc.state, np.array([0, 0, 1, 0, 0, 0, 0, 0])), "Improper Pauli-X functionality"

## Multi qubit gates

These gates typically have one or more control gate and a target gate.

Example from: https://quantumcomputing.stackexchange.com/a/24209

$CNOT_{1,3} = |0\rangle\langle0| \otimes I \otimes I + |1\rangle\langle1| \otimes I \otimes X$

In [44]:
class QCircuit(QCircuit):
    def cnot(self, control, target):
        # Make |0><0| ⊗ I ⊗ I
        operation1 = [(control, np.outer([1, 0], [1, 0]))]
        gate1 = self.compose(operation1)
        # Make |1><1| ⊗ I ⊗ X
        operation2 = [(control, np.outer([0, 1], [0, 1]))]
        operation2.append((target, np.array([[0, 1], [1, 0]])))
        gate2 = self.compose(operation2)
        # Combine operations and apply to state vector
        self.state = np.dot(gate1 + gate2, self.state)

# Apply CNOT to |01> to yield |11>
qc = QCircuit(2)
qc.px(0)
qc.cnot(0, 1)
assert np.allclose(qc.state, np.array([0, 0, 0, 1]))